## 1. 사전환경 구성

In [16]:
# import librarys
from google.colab import output
from google.colab import drive
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
from PIL import Image
from sklearn.model_selection import KFold
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset

In [17]:
# mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [19]:
# set path
os.chdir('/content/drive/MyDrive/dirty_mnist')
ROOT_PATH = Path(os.getcwd())
print(f'ROOT_PATH : {ROOT_PATH}')

ROOT_PATH : /content/drive/MyDrive/dirty_mnist


In [20]:
# set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device : {device}')

device : cuda:0


## 2. 데이터셋 구성

In [21]:
# define ToTensor
class ToTensor(object) :
    '''
    numpy array를 torch tensor로 변환
    변환하고자 하는 array를 전달인자로 하여 호출 (np.ndarray)
    image의 차원을 tensor 형식으로 변경 (C x H x W)
    '''
    
    def __call__(self, sample) :
        image, label = sample['image'], sample['label']
        image = image.transpose((2, 0, 1))
        return {'image': torch.FloatTensor(image),
                'label': torch.FloatTensor(label)}

In [22]:
# create transform pipline object
train_transforms = T.Compose([ToTensor()])
test_transforms = T.Compose([ToTensor()])

In [23]:
# define DatasetMNIST
class DatasetMNIST(torch.utils.data.Dataset) :
    '''
    DataLoader에 의해 호출 될 Dataset
    dif_path : image가 위치한 directory path
    meta_df : train data의 answers
    __getitem__.index : DataLoader가 입력
    __getitem__ return : dict type object (key = ['image', 'label'])
    '''
    
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=None,
                 augmentations=None
                 ) :

        self.dir_path = dir_path
        self.meta_df = meta_df
        self.transforms = transforms
        self.augmentations = augmentations
    
    def __len__(self) :
        return len(self.meta_df)
    
    def __getitem__(self, index) :
        image = cv2.imread(self.dir_path+\
                           str(self.meta_df.iloc[index,0]).zfill(5)+'.png',
                           cv2.IMREAD_GRAYSCALE)
        image = (image/255).astype('float')[..., np.newaxis]
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        sample = {'image': image, 'label': label}
        if self.transforms :
            sample = self.transforms(sample)
        
        return sample

In [24]:
# load data labels
mnist_answers = pd.read_csv(ROOT_PATH/'data/dirty_mnist_2nd_answer.csv')
mnist_answers.head()

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,2,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,3,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,4,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0


## 3. 모델 구성

In [25]:
# define MultiLabelResnet18
class MultiLabelResnet18(nn.Module) :
    '''
    Pretrained ResNet18 사용
    Input과 Output Dimension을 맞추기 위해 layer 추가
    최종 출력은 softmax가 아닌 sigmoid 사용
    '''

    def __init__(self) :
        super(MultiLabelResnet18, self).__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self.resnet = models.resnet18(pretrained=True)
        self.FC = nn.Linear(1000, 26)
    
    def forward(self, x) :
        x = F.relu(self.conv2d(x))
        x = F.relu(self.resnet(x))
        x = torch.sigmoid(self.FC(x))
        return x

In [26]:
# define new_model
def new_model(model=MultiLabelResnet18, lr=0.001, step_size=5, gamma=0.75) :
    model = model()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=step_size, gamma=gamma)
    
    return model, optimizer, lr_scheduler

## 4. 모델 학습 및 평가

In [27]:
# set training schedule
n_fold = 3
n_epochs = 3
train_batch_size = 128
valid_batch_size = 32

In [28]:
# initialize objects
kfold = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
best_models = []
train_data_path = str(ROOT_PATH)+'/data/train_data/'

In [ ]:
# loop in kfold
for fold_count, (train_index, valid_index) in enumerate(kfold.split(mnist_answers), 1) :
    print(f'[fold: {fold_count}]')
    torch.cuda.empty_cache()
    
    # create data iterator 
    train_answer = mnist_answers.iloc[train_index]
    valid_answer = mnist_answers.iloc[valid_index]

    train_dataset = DatasetMNIST(
        dir_path = train_data_path,
        meta_df = train_answer,
        transforms = train_transforms
    )
    valid_dataset = DatasetMNIST(
        dir_path = train_data_path,
        meta_df = valid_answer,
        transforms = train_transforms
    )
    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size = train_batch_size,
        shuffle = False,
        num_workers = 3
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = valid_batch_size,
        shuffle = False,
        num_workers = 3
    )

    # create new model, optimizer, etc
    model, optimizer, lr_scheduler = new_model()
    criterion = torch.nn.BCELoss()

    valid_acc_max = 0
    for epoch in range(n_epochs) :
        
        # start to training
        train_acc_list = []
        with tqdm(train_data_loader,
                  total=train_data_loader.__len__(),
                  unit="batch"
                  ) as train_bar :
            for sample in train_bar :
                train_bar.set_description(f"Train Epoch {epoch}")
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                model.train()
                with torch.set_grad_enabled(True) :
                    probs = model.forward(images)
                    loss_out = criterion(probs, labels)
                    loss_out.backward()
                    optimizer.step()

                probs = probs.cpu().detach().numpy()
                labels = labels.cpu().detach().numpy()
                preds = probs > 0.5
                batch_acc = (labels == preds).mean()
                train_acc_list.append(batch_acc)
                train_acc = np.mean(train_acc_list)
                train_bar.set_postfix(train_loss = loss_out.item(),
                                        train_acc = train_acc)
        
        # start to validation
        valid_acc_list = []
        with tqdm(valid_data_loader,
                  total=valid_data_loader.__len__(),
                  unit="batch"
                  ) as valid_bar :
            for sample in valid_bar :
                valid_bar.set_description(f"Valid Epoch {epoch}")
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                model.eval()
                with torch.no_grad() :
                    probs = model.forward(images)
                    loss_out = criterion(probs, labels)
                    loss_out.backward()
                    optimizer.step()

                probs = probs.cpu().detach().numpy()
                labels = labels.cpu().detach().numpy()
                preds = probs > 0.5
                batch_acc = (labels == preds).mean()
                valid_acc_list.append(batch_acc)
                valid_acc = np.mean(valid_acc_list)
                valid_bar.set_postfix(valid_loss = loss_out.item(),
                                        valid_acc = valid_acc)

        # learning rate scheduling
        lr_scheduler.step()

        # save good models
        if valid_acc_max < valid_acc :
            valid_acc_max = valid_acc
            best_model = model
            MODEL = 'resnet18'
            model_path = str(ROOT_PATH)+'/model'
            torch.save(best_model, f'{model_path}{fold_index}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}.pth')
    
    best_models.append(best_model)

[fold: 1]


Train Epoch 0:  31%|███       | 81/261 [04:57<22:28,  7.49s/batch, train_acc=0.541, train_loss=0.682]

## 5. 테스트 환경 구성

In [ ]:
# set test dataset
test_data_path = str(ROOT_PATH)+'/data/test_data'
test_batch_size =128

sample_submission = pd.read_csv(ROOT_PATH/'data/sample_submission.csv')
test_dataset = DatasetMNIST(
    dir_path = test_data_path,
    meta_df = sample_submission,
    transforms = test_transforms
)
test_data_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

In [ ]:
# initialize objects
predictions_list = []
prediction_df = pd.read_csv(ROOT_PATH/'data/sample_submission.csv')

## 6. 테스트 수행

In [ ]:
# loop in best models
for model in best_models :
    prediction_array = np.zeros([prediction_df.shape[0], 
                                 prediction_df.shape[1]-1])
    # inference about test dataset
    for idx, sample in enumerate(test_data_loader) :
        with torch.no_grad() :
            model.eval()
            images = sample['image']
            images = images.to(device)
            probs = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)
            
            batch_index = test_batch_size * idx
            prediction_array[batch_index : batch_index + images.shape[0], : ] = preds.astype(int)
    
    # save every model's prediction
    predictions_list.append(prediction_array[...,np.newaxis])

## 7. 앙상블 및 제출파일 생성

In [ ]:
# ensemble
predictions_array = np.concatenate(prdictions_list, axis = 2)
predictions_mean = predictions_array.mean(axis = 2)
predictions_mean = (predictions_mean > 0.5) * 1

In [ ]:
# save result file
file_name = str(model_path)+f'/{MODEL}_result.csv'

prediction_df.iloc[:, 1:] =  predictions_mean
predcition_df.to_csv(file_name, index = False)

print('Done')